
# Multiple signals from same single-cell simulation

In [ ]:
%matplotlib inline
import os
from os.path import join
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
from matplotlib.patches import Ellipse
import elephant
import LFPy
import neuron
from neuron import h
from lfpykit.eegmegcalc import (FourSphereVolumeConductor, 
                                SphericallySymmetricVolCondMEG)
from LFPy import InfiniteHomogeneousVolCondMEG
import brainsignals.neural_simulations as ns
from brainsignals.plotting_convention import mark_subplots, simplify_axes, color_axes

neuron.load_mechanisms(ns.cell_models_folder)
np.random.seed(12345)

tstop = 200
dt = 2**-5
sigma = 0.3


In [ ]:
def insert_synaptic_input(cell, synidx, spiketime, syn_e, weight_factor):

    synapse_parameters = dict(
                          idx = synidx,
                          e = syn_e, # reversal potential
                          weight = 0.005 * weight_factor, # synapse weight
                          record_current = False, # record synapse current
                          syntype = 'Exp2Syn',
                          tau1 = 0.1, #Time constant, rise
                          tau2 = 1.0, #Time constant, decay
                          )
    synapse = LFPy.Synapse(cell, **synapse_parameters)
    synapse.set_spike_times(np.array([spiketime]))
    return synapse, cell


cell = ns.return_hay_cell(tstop=tstop, dt=dt, make_passive=False)
ns.point_axon_down(cell)

# Uniform background excitation
num_syns = 100
synidxs = cell.get_rand_idx_area_norm(section='allsec', 
                                      z_max=1e9, z_min=-1e9, nidx=num_syns)
syn_times = np.random.uniform(0, tstop, size=num_syns)
syn_e = 0
weight_factor = 1
for idx in range(num_syns):
    synapse, cell = insert_synaptic_input(cell, synidxs[idx], syn_times[idx], 
                                          syn_e, weight_factor)

num_syns = 200
synidxs = cell.get_rand_idx_area_norm(section='allsec', 
                                      z_max=1e9, z_min=800, nidx=num_syns)
syn_times = np.random.normal(50, 15, size=num_syns)
syn_e = 0
weight_factor = 1
for idx in range(num_syns):
    synapse, cell = insert_synaptic_input(cell, synidxs[idx], syn_times[idx], 
                                          syn_e, weight_factor)

num_syns = 200
synidxs = cell.get_rand_idx_area_norm(section='allsec', 
                                      z_max=800, z_min=-1e9, nidx=num_syns)
syn_times = np.random.normal(50, 15, size=num_syns)
syn_e = -90
weight_factor = 1
for idx in range(num_syns):
    synapse, cell = insert_synaptic_input(cell, synidxs[idx], syn_times[idx], 
                                          syn_e, weight_factor)

num_syns = 200
synidxs = cell.get_rand_idx_area_norm(section='allsec', 
                                      z_max=1e9, z_min=800, nidx=num_syns)
syn_times = np.random.normal(100, 15, size=num_syns)
syn_e = -90
weight_factor = 1
for idx in range(num_syns):
    synapse, cell = insert_synaptic_input(cell, synidxs[idx], syn_times[idx], 
                                          syn_e, weight_factor)

num_syns = 200
synidxs = cell.get_rand_idx_area_norm(section='allsec', 
                                      z_max=800, z_min=-1e9, nidx=num_syns)
syn_times = np.random.normal(100, 15, size=num_syns)
syn_e = 0
weight_factor = 0.25
for idx in range(num_syns):
    synapse, cell = insert_synaptic_input(cell, synidxs[idx], syn_times[idx], 
                                          syn_e, weight_factor)
    

cell.simulate(rec_imem=True, rec_vmem=True)


In [ ]:
# LFP and EAP:
filt_dict_hf = {'highpass_freq': 300,
                 'lowpass_freq': None,
                 'order': 4,
                 'filter_function': 'filtfilt',
                 'fs': 1 / dt * 1000,
                 'axis': -1}

filt_dict_lf = {'highpass_freq': None,
                 'lowpass_freq': 300,
                 'order': 4,
                 'filter_function': 'filtfilt',
                 'fs': 1 / dt * 1000,
                 'axis': -1}

elec_params = dict(
            sigma = sigma,      # extracellular conductivity
            x = np.array([-20, 100]),
            y = np.zeros(2),
            z = np.array([0, 800]),
            method = 'root_as_point',
        )
elec = LFPy.RecExtElectrode(cell, **elec_params)
M_elec = elec.get_transformation_matrix()

v_e = M_elec @ cell.imem * 1000
v_e -= v_e[:, 0, None]

eap = elephant.signal_processing.butter(v_e[0], **filt_dict_hf)
lfp = elephant.signal_processing.butter(v_e[1], **filt_dict_lf)

# EEG
radii = [89000., 90000., 95000., 100000.]  # (µm)
sigmas = [0.276, 1.65, 0.01, 0.465]  # (S/m)
eps = 1e-2
x = np.array([0])
y = np.array([0])
z = np.array([radii[-1] - eps])

r_elecs = np.array([x, y, z]).T # (µm)
sphere_model = FourSphereVolumeConductor(r_elecs, radii, sigmas)
cdm = LFPy.CurrentDipoleMoment(cell).get_transformation_matrix() @ cell.imem

somapos = np.array([0., 0., radii[0] - np.max(cell.z) - 100])
r_soma_syns = [cell.get_intersegment_vector(idx0=0, idx1=i)
               for i in cell.synidx]
r_mid = np.average(r_soma_syns, axis=0)
r_mid = somapos + r_mid/2.

M_eeg = sphere_model.get_transformation_matrix(r_mid)  
eeg = elephant.signal_processing.butter(1e6 * M_eeg @ cdm, **filt_dict_lf) # (uV)


# MEG
# mulitply by µ_0 to get B
mu_0 = 4*np.pi*10**-7 # [Tm/A]
# nA/µm * Tm/A = mT (millitesla)
# multiply by scaling factor to get femtoTesla
k = 1e12 # from mT to fT
# spherically symmetric volume conductor model
meg_sphere = SphericallySymmetricVolCondMEG(r=r_elecs)
# linear mapping between current dipole moment and signal at sensor locations
M_meg = meg_sphere.get_transformation_matrix(r_p=r_mid)
B_meg = elephant.signal_processing.butter(mu_0 * k * M_meg @ cdm, **filt_dict_lf) #  [fT]


# ECoG
ecog_elec_params = dict(
            sigma = sigma,      # extracellular conductivity
            x = np.array([0]),
            y = np.array([0]),
            z = np.array([np.max(cell.z) + 20]),
            method = 'root_as_point',
            N = [[0, 0, 1]],
            r = 150,
            n = 500,
        )

ecog_elec = LFPy.RecExtElectrode(cell, **ecog_elec_params)
M_ecog = ecog_elec.get_transformation_matrix()
v_e = M_ecog @ cell.imem * 1000 * 2
v_e -= v_e[:, 0, None]
ecog = elephant.signal_processing.butter(v_e[0], **filt_dict_lf)

# Local magnetic field B assuming infinite homogeneous volume conductor
sensor_locations = np.c_[elec_params["x"][1],
                         elec_params["y"][1], 
                         elec_params["z"][1]]
mag_loc = InfiniteHomogeneousVolCondMEG(sensor_locations)
H_local = mag_loc.calculate_H_from_iaxial(cell)
B_local = elephant.signal_processing.butter(H_local * mu_0 * k, **filt_dict_lf)


In [ ]:
eeg_clr = 'royalblue'
ecog_clr = 'royalblue'
eap_clr = 'royalblue'
lfp_clr = 'royalblue'
meg_clr = 'seagreen'
mag_clr = 'seagreen'
vm_clr = 'firebrick'

xmin = -155
xmax = 220
zmin = -160
zmax = 1150

def plot_four_sphere_model(ax, radii):
    head_colors = ["#ffb380", "#74abff", "#b3b3b3", "#c87137"]
    for i in range(4):
        ax.add_patch(plt.Circle((0, 0), radius=radii[-1 - i],
                                   color=head_colors[-1-i],
                                   fill=True, ec='k', lw=.1))
    # mark 4-sphere head model layers
    ax.text(26000, radii[0] - 6000, 'brain', ha="left", va="top", rotation=-0)
    ax.text(26000, radii[1] - 3000, 'CSF', ha="left", va="top", rotation=-0)
    ax.text(26000, radii[2] - 3000, 'skull', ha="left", va="top", rotation=-0)
    ax.text(26000, radii[3] - 3000, 'scalp', ha="left", va="top", rotation=-0)

fig = plt.figure(figsize=[4, 4])
fig.subplots_adjust(bottom=0.05, top=0.98, left=0.01, right=0.99, wspace=0.2)

ax_m = fig.add_axes([0.3, 0.05, 0.4, 0.6], frameon=False, aspect=1,
                        xticks=[], yticks=[], xlim=[xmin, xmax],
                        ylim=[zmin, zmax])

ax_4s = fig.add_axes([0.42, 0.78, 0.5, 0.25], frameon=False, aspect=1,
                    xticks=[], yticks=[], xlim=[-25000, 25000], 
                     ylim=[75000, 110000])

plot_four_sphere_model(ax_4s, radii)

ax_vm = fig.add_axes([0.77, 0.08, 0.2, 0.1], title="membrane\npotential", xlabel="time (ms)", ylabel="mV")
ax_eap = fig.add_axes([0.13, 0.08, 0.2, 0.1], title="spike", xlabel="time (ms)", ylabel="µV")
ax_lfp = fig.add_axes([0.13, 0.35, 0.2, 0.1], title="LFP", xlabel="time (ms)", ylabel="µV")
ax_ecog = fig.add_axes([0.13, 0.59, 0.2, 0.1], title="ECoG", xlabel="time (ms)", ylabel="µV")
ax_eeg = fig.add_axes([0.13, 0.84, 0.2, 0.1], title="EEG", xlabel="time (ms)", ylabel="nV")
ax_meg = fig.add_axes([0.77, 0.59, 0.2, 0.1], title="MEG", xlabel="time (ms)", ylabel="B$_x$ (fT)")
ax_mag = fig.add_axes([0.77, 0.35, 0.2, 0.1], title="magnetic field", xlabel="time (ms)", ylabel="B$_x$ (fT)")


color_axes(ax_vm, vm_clr)
color_axes(ax_eap, eap_clr)
color_axes(ax_lfp, lfp_clr)
color_axes(ax_ecog, ecog_clr)
color_axes(ax_eeg, eeg_clr)
color_axes(ax_meg, meg_clr)
color_axes(ax_mag, mag_clr)

t1 = 20
t1_idx = np.argmin(np.abs(cell.tvec - t1))

#tlim_eap = [cell.tvec[np.argmin(eap)] - 2, cell.tvec[np.argmin(eap)] + 3]
#ax_eap.set_xlim(tlim_eap)

num_elecs = len(elec.x)
max_v_e = np.max(np.abs(v_e))

ax_m.plot(cell.x.T, cell.z.T, c='k', clip_on=False)
ax_4s.plot(cell.x.T, cell.z.T + radii[0] - np.max(cell.z) - 20, c='k', clip_on=False, lw=0.7)

ax_eap.plot(cell.tvec, eap, c=eap_clr, lw=1.)
ax_lfp.plot(cell.tvec, lfp, c=lfp_clr, lw=1.)
ax_eeg.plot(cell.tvec, eeg[0], c=eeg_clr, lw=1.)
ax_ecog.plot(cell.tvec, ecog, c=ecog_clr, lw=1.)
ax_meg.plot(cell.tvec, B_meg[0][0], c=meg_clr, lw=1.)
ax_mag.plot(cell.tvec, B_local[0][0], c=mag_clr, lw=1.)

ax_vm.plot(cell.tvec, cell.vmem[0, :], c=vm_clr, lw=1.)
    
ax_m.plot(cell.x[0].mean(), cell.z[0].mean(), 'o', c=vm_clr, ms=4)
ax_m.plot([cell.x[0].mean(), cell.x[0].mean()- 350], 
          [cell.z[0].mean(), cell.z[0].mean() - 25], 
          ":", clip_on=False, c=eap_clr, lw=0.7)

ax_m.plot(elec.x[0], elec.z[0], 'o', c=eap_clr, ms=4)
ax_m.plot([elec.x[0], elec.x[0] + 350], 
          [elec.z[0], elec.z[0] + 1], 
          ":", clip_on=False, c=vm_clr, lw=0.7)
    
ax_m.plot(elec.x[1], elec.z[1], 'o', c=lfp_clr, ms=5, clip_on=False)
ax_m.plot([elec.x[1], elec.x[1] + 450], 
          [elec.z[1], elec.z[1] - 50], 
          ":", clip_on=False, c=mag_clr, lw=0.7)

ax_m.plot(elec.x[1], elec.z[1], 'o', c=mag_clr, ms=2.5)
ax_m.plot([elec.x[1], elec.x[1] - 500], 
          [elec.z[1], elec.z[1] - 150], 
          ":", clip_on=False, c=lfp_clr, lw=0.7)
    
el = Ellipse((ecog_elec.x[0], ecog_elec.z[0]), 
             2*ecog_elec_params["r"], ecog_elec_params["r"] / 8,
             facecolor=ecog_clr, clip_on=False)
ax_m.add_artist(el)
ax_m.plot([ecog_elec.x[0] - 150, ecog_elec.x[0] - 350], 
          [ecog_elec.z[0], ecog_elec.z[0] - 20], 
          ":", clip_on=False, c=ecog_clr, lw=0.7)

el2 = Ellipse((r_elecs[0, 0], r_elecs[0, 2]), 
             10000, 1000, edgecolor=eeg_clr, lw=1,
             facecolor=meg_clr, clip_on=False)
ax_4s.add_artist(el2)

ax_4s.plot([r_elecs[0, 0] - 5000, r_elecs[0, 0] - 45000], 
          [r_elecs[0, 2], r_elecs[0, 2] - 2000], 
          ":", clip_on=False, c=eeg_clr, lw=0.7)

# MEG location line
ax_4s.plot([r_elecs[0, 0] + 500, r_elecs[0, 0] + 20000], 
          [r_elecs[0, 2] - 500, r_elecs[0, 2] - 40000], 
          ":", clip_on=False, c=meg_clr, lw=0.7)

# Cell inset location lines
z0 = radii[0] - np.max(cell.z) - 200
ax_4s.plot([-700, 700, 700, -700, -700], 
           np.array([-1000, -1000, 1500, 1500, -1000]) + z0, 
           lw=0.5, c='gray', ls='--')

ax_4s.plot([-700, -700 - 40000],
           [-1000 + z0, + z0 - 22000], 
           lw=0.5, c='gray', ls="--", clip_on=False)

ax_4s.plot([+700, +700 - 11000],
           [-1000 + z0, + z0 - 24000], 
           lw=0.5, c='gray', ls="--", clip_on=False)

el3 = Ellipse((0, np.max(cell.z) + 100), 
             500, 100, edgecolor='gray', ls='--', lw=0.5,
             facecolor='none', clip_on=False)
ax_m.add_artist(el3)


simplify_axes(fig.axes)
fig.savefig(join("hay_multisignal.pdf"))